# Титаник (ДЗ3) Дерево решений

## Загрузим выборку

In [162]:
import pandas as pd
import numpy as np
import re

In [155]:
titanic_data_train = pd.read_csv('train.csv')
titanic_data_test = pd.read_csv('test.csv')

## Создадим единый фрейм для обеих выборок

In [156]:
titanic_data = titanic_data_train.append(titanic_data_test)

## Пропуски данных

## Предобработка данных

## Заполним Age

In [20]:
titanic_data.reset_index(drop=True, inplace=True)

In [158]:
titanic_data['Prefix'] = [re.sub('[, .]', '',re.search(r', \w+.', name).group()) for name in titanic_data['Name']]
mean_master_age = titanic_data[titanic_data['Prefix'] == 'Master']['Age'].mean()
titanic_data.loc[(titanic_data['Age'].isnull()) & (titanic_data['Prefix'] == 'Master'), 'Age'] = mean_master_age
titanic_data['Age'].fillna(titanic_data['Age'].mean(), inplace = True)

## Заполним пропуски в Embarked

In [24]:
titanic_data[titanic_data['Embarked'].isnull()]

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,Prefix
61,38.0,B28,NaN,80.0,"Icard, Miss. Amelie",0,62,1,female,0,1.0,113572,Miss
829,62.0,B28,NaN,80.0,"Stone, Mrs. George Nelson (Martha Evelyn)",0,830,1,female,0,1.0,113572,Mrs


In [26]:
titanic_data.loc[titanic_data['Embarked'].isnull(), 'Embarked'] = 'S'

## Заполним пропуски в Fare

In [28]:
titanic_data[titanic_data['Fare'].isnull()]

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,Prefix
1043,60.5,NaN,S,NaN,"Storey, Mr. Thomas",0,1044,3,male,0,NaN,3701,Mr


In [30]:
third_class_mean_fare = titanic_data[titanic_data['Pclass'] == 3]['Fare'].mean()
third_class_mean_fare

13.302888700564969

In [31]:
titanic_data.loc[titanic_data['Fare'].isnull(), 'Fare'] = third_class_mean_fare

Данные заполнены для всех полей за исключением Cabin(пока не будем использовать для предсказания) и Survived - целевая переменная

## Создадим дерево решений

In [152]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(min_samples_leaf=5, max_depth=5)
clf.fit(titanic_data[np.isfinite(titanic_data['Survived'])].loc[:,['Fare', 'Age', 'Pclass']], 
        titanic_data[np.isfinite(titanic_data['Survived'])]['Survived'])

predicted = clf.predict(titanic_data[np.isnan(titanic_data['Survived'])].loc[:,['Fare', 'Age', 'Pclass']])
survival = 'PassengerId,Survived\n'
for row, index in enumerate(titanic_data[np.isnan(titanic_data['Survived'])].index):
    survival += '{},{}\n'.format(titanic_data.loc[index, 'PassengerId'], int(predicted[row]))

In [151]:
with open('prediction5.txt', 'w') as outfile:
    outfile.write(survival)

## Результаты изменения параметров min_samples_leaf, max_depth